In [ ]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
import dotenv
import os

dotenv.load_dotenv()

os.environ["GROQ_API_KEY"] = "xyz"

In [44]:

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [45]:
@tool
def add(a:int, b:int) -> int:
    """Add two numbers"""
    return a + b

In [46]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [47]:
multiply.name

'multiply'

In [48]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [49]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

### tool binding

In [50]:

llm = ChatGroq(
    model_name="openai/gpt-oss-120b",
    temperature=0.7
)

In [51]:
llm.invoke('hi')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "hi". The system says we need to respond as ChatGPT. There\'s no policy violation. Just greet.'}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 72, 'total_tokens': 116, 'completion_time': 0.09281679, 'completion_tokens_details': {'reasoning_tokens': 26}, 'prompt_time': 0.003133464, 'prompt_tokens_details': None, 'queue_time': 0.055127976, 'total_time': 0.095950254}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bda68-d165-7430-8c20-2f7e7d377195-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 72, 'output_tokens': 44, 'total_tokens': 116, 'output_token_details': {'reasoning': 26}})

In [52]:
llm_with_tools = llm.bind_tools([multiply,add])

In [53]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="Hello! I'm doing great, thank you for asking. How can I assist you today?", additional_kwargs={'reasoning_content': 'The user says "Hi how are you". This is a casual greeting. We need to respond politely. No special instructions. Just respond.'}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 155, 'total_tokens': 211, 'completion_time': 0.121614448, 'completion_tokens_details': {'reasoning_tokens': 29}, 'prompt_time': 0.00665526, 'prompt_tokens_details': None, 'queue_time': 0.0545456, 'total_time': 0.128269708}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_a09bde29de', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bda68-d3b3-7b32-b814-b55ec6ef6b74-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 155, 'output_tokens': 56, 'total_tokens': 211, 'output_token_details': {'reasoning': 29}})

In [63]:
query = HumanMessage('can you add 3 with 1000 and then multiply 5 to 10 , i want two different answers of twor seperate funcitons')

In [64]:
messages = [query]

In [65]:
result = llm_with_tools.invoke(messages)
result # as we can see here , when we ask any question releated to the working of the tool binded with llm , then in output of llm content will be empty ='', and 'tool_calls' has all the answer

AIMessage(content='', additional_kwargs={'reasoning_content': 'The user wants to add 3 with 1000 and then multiply 5 to 10, i.e., compute 3+1000 = 1003, and 5*10 = 50. They want two different answers of two separate functions. We have two functions: add and multiply. We should call add with a=3, b=1000, and multiply with a=5, b=10. Then present both results. Use the functions tool.', 'tool_calls': [{'id': 'fc_1cb56e9e-36c7-48c6-bb3f-20b3dd89c70d', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 180, 'total_tokens': 312, 'completion_time': 0.280073744, 'completion_tokens_details': {'reasoning_tokens': 98}, 'prompt_time': 0.006809098, 'prompt_tokens_details': None, 'queue_time': 0.156682671, 'total_time': 0.286882842}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_5a93aea882', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'mod

In [66]:
result.tool_calls

[{'name': 'add',
  'args': {'a': 3, 'b': 1000},
  'id': 'fc_1cb56e9e-36c7-48c6-bb3f-20b3dd89c70d',
  'type': 'tool_call'}]

In [58]:
messages.append(result)

In [59]:
messages

[HumanMessage(content='can you add 3 with 1000 and multiply the answer to 100', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': 'User wants to add 3 and 1000, then multiply the answer by 100. So compute (3+1000)*100 = 1003*100 = 100300.\n\nWe can use functions add and multiply.\n\nFirst add 3 and 1000 => 1003. Then multiply 1003 by 100 => 100300.\n\nWe should call add then multiply.', 'tool_calls': [{'id': 'fc_824d4819-e194-4d89-b07e-6de933f8e352', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 115, 'prompt_tokens': 167, 'total_tokens': 282, 'completion_time': 0.248516027, 'completion_tokens_details': {'reasoning_tokens': 81}, 'prompt_time': 0.037137368, 'prompt_tokens_details': None, 'queue_time': 0.060941472, 'total_time': 0.285653395}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_626f3fc5e0', 'service_tier': 'on_d

In [61]:
tool_result = multiply.invoke(result.tool_calls[0]) # from results, tool_calls[0] , {'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'fc_18a081db-b129-4729-b3ca-0cbfe7f74bbe', 'type': 'tool_call'} this dict is given to multiply tool

In [62]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='fc_824d4819-e194-4d89-b07e-6de933f8e352')

In [ ]:
messages.append(tool_result)


In [ ]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'reasoning_content': "We need to multiply 3 and 1000 using the provided function. We'll call multiply with a=3, b=1000.", 'tool_calls': [{'id': 'fc_5bbfd970-9d2a-4d51-af42-d99c32ca0f3f', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 138, 'total_tokens': 201, 'completion_time': 0.13270072, 'completion_tokens_details': {'reasoning_tokens': 29}, 'prompt_time': 0.005213738, 'prompt_tokens_details': None, 'queue_time': 0.009605993, 'total_time': 0.137914458}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_549ac91920', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd627-9673-7a50-93d5-40ae03ece833-0', tool_calls=[{'name': 'multiply', 'args': {'a'

In [ ]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is **3000**.'